我们用一个例子来体验一下istio Service Mesh。现在我们有gke-us-west1 和gke-eu-west1 两个集群，并且在playground namespace 设置了istio-injection: enabled

In [14]:
kubectl get namespace playground -o yaml

apiVersion: v1
kind: Namespace
metadata:
  annotations:
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"v1","kind":"Namespace","metadata":{"annotations":{},"labels":{"argocd.argoproj.io/instance":"project-access-gke-us-west1","findi.io/plan":"free","istio-injection":"enabled","name":"playground"},"name":"playground"}}
  creationTimestamp: "2022-08-31T20:50:18Z"
  labels:
    argocd.argoproj.io/instance: project-access-gke-us-west1
    findi.io/plan: free
    istio-injection: enabled
    kubernetes.io/metadata.name: playground
    name: playground
  name: playground
  resourceVersion: "26231392"
  uid: 28b5d39f-c298-473b-9db2-8da5bece515c
spec:
  finalizers:
  - kubernetes
status:
  phase: Active


首先使用helm 安装nginx，参考https://artifacthub.io/packages/helm/bitnami/nginx

In [ ]:
helm repo add bitnami https://charts.bitnami.com/bitnami

In [15]:
echo " 
replicaCount: 1
cloneStaticSiteFromGit:
enabled: true
repository: https://github.com/mdn/beginner-html-site-styled.git
branch: master
service:
type: ClusterIP
ingress:
enabled: true
annotations:
  kubernetes.io/tls-acme: "true"
  cert-manager.io/cluster-issuer: issuer-nginx
  external-dns: "true"
  nginx.ingress.kubernetes.io/service-upstream: "true"
  nginx.ingress.kubernetes.io/upstream-vhost: "nginx.playground.svc.cluster.local"
ingressClassName: "nginx"
hostname: "nginx-us.playground.findi.biz"
tls: true
" > bitnami.yaml

In [ ]:
helm upgrade --install -n playground nginx bitnami/nginx --values bitnami.yaml

检查一下安装状态

In [ ]:
helm list -n playground

In [ ]:
kubectl get pods -n playground

In [ ]:
kubectl get ingress -n playground

接下来我们访问一下链接来验证一下, 可能需要几分钟等待DNS更新 https://nginx-us.playground.findi.biz/ .尝试刷新一下页面，你会发现页面显示不同内容， 因为在另外一个集群gke-eu-west1上部署了不同的内容。

现在尝试把当前集群的pod停止

In [ ]:
kubectl scale deployment -n playground nginx --replicas=0

检查一下pod状态，你会发现没有pod在运行

In [ ]:
kubectl get pods -n playground

这时候再次访问上面的链接https://nginx-us.playground.findi.biz/ 仍然能够访问，因为所有traffic会被转到状态正常的Pod